## Notebook 11.1: *denovo* Genome assembly 


### Learning objectives: 

By the end of this notebook you will:

+ Be familiar with assembled genome file formats. 
+ Know where to find assembled genomes and their raw files online.
+ Be able to compute genome size from kmer counts of Illumina short reads. 
+ Experience running Illumina, and Illumina+PacBio hybrid *de novo* assemblies.  

### Assigned readings: 

**Read these papers carefully, we will discuss in class:**

+ Li, Fay-Wei, and Alex Harkess. “A Guide to Sequence Your Favorite Plant Genomes.” Applications in Plant Sciences 6, no. 3 (n.d.): e1030. https://doi.org/10.1002/aps3.1030.



+ Sumpter, Nicholas, Margi Butler, and Russell Poulter. “Single-Phase PacBio De Novo Assembly of the Genome of the Chytrid Fungus Batrachochytrium Dendrobatidis, a Pathogen of Amphibia.” Microbiology Resource Announcements 7, no. 21 (November 2018). https://doi.org/10.1128/MRA.01348-18.



**Please skim the paper below in terms of the abstract, figures, and tables.**
+ Giordano, Francesca, Louise Aigrain, Michael A. Quail, Paul Coupland, James K. Bonfield, Robert M. Davies, German Tischler, et al. “De Novo Yeast Genome Assemblies from MinION, PacBio and MiSeq Platforms.” Scientific Reports 7, no. 1 (June 21, 2017): 3935. https://doi.org/10.1038/s41598-017-03996-z.




### Software used in this notebook
We will use three Python packages and also a number of command-line programs that have been installed using the conda package manager. The commands to install the software are shown below for reference, but are commented out. You do not need to run the installation commands since the software is already installed for you. 

In [1]:
import toyplot
import numpy as np
import pandas as pd

In [2]:
# conda install numpy pandas
# conda install eaton-lab::toyplot   
# conda install bioconda::cutadapt      # read adapter/quality trimmer
# conda install bioconda::jellyfish     # kmer counting
# conda install bioconda::spades        # debruijn graph assembler
# conda install bioconda::minimap2      # short/long read mapper/aligner
# conda install bioconda::miniasm       # OLC-based de novo assembler
# conda install bioconda::racon         # polisher: consensus calls on long read assembly   

### Computational resources for genome assembly

Genome assembly is very computationally intensive. Specifically, *de novo* genome assembly -- the process of assembling a genome without any *a priori* guide. This stands in contrast to reference-guided assembly methods which are used to assemble a genome by mapping reads to an existing close relative's genome. The *de novo* process is incredibly more demanding both in terms of the amount of input data and computational power that is required. 

In particular, *de novo* assembly requires enormous amounts of RAM (i.e., accessible memory). This type of resource is not easily shared across nodes of a computing cluster (e.g., a University HPC system), and so instead clusters typically contain a few special nodes, designated *high-memory* nodes, that are  best for this process. As an example, even small bacterial genomes will typically require 8GB or more of RAM, and human genomes require 100-500Gb, depending on the software being used. Much larger genomes obviously require even more. High memory nodes often have 1Tb of RAM or more. 

In this notebook and the next we will complete several genome assemblies using real data. However, to avoid the need for significant computational resources most of the computationally intensive code is commented out, meaning you will not actually run it. We've made all of the files available to you so that you can see the results that are produced, and look at the files to interpret their format and meaning. 

For these exercises we will work with a relatively small genome and a smallish number of sequenced reads with the goal of assembling the Eukaryotic genome *Batrachochytrium dendrobatidis*, a Chytrid fungus that is an ecologically important pathogen of amphibians, and the subject of one of your assigned readings. 

## The *Bd* genome
We will assemble the *Bd* chytrid fungus genome from the assigned Sumpter et al. (2018) paper. I selected this paper because it provides a very concise report of a genome assembly and also where to find the associated data.

The first draft genome assembly of *Bd* genome was sequenced by the US DOE Joint Genome Institute using Sanger sequencing to 8.74X coverage in 2011. That assembly is in 127 scaffolds (N50 scaffold size of 1.48Mb) and 510 contigs (N50 contig size 318K) (https://www.ncbi.nlm.nih.gov/assembly/GCF_000203795.1). In your assigned p aper by Sumpter et al. a new draft genome is assembled using PacBio data alone (and some additional scaffolding methods). Our exercise in this notebook will be to re-assemble this data set using their data, and to explore alternative assembly methods. 


<div class="alert alert-success">
    <b>[1] Question:</b> 
    What are the assembly statistics for the initial draft genome assembled by Sumpter et al. using PacBio data (before they do additional scaffolding)? Answer in markdown below.  
</div>

In [3]:
#answer here:   


## Download sequence data (.fastq) files

The paper by Sumpter et al. includes a section called *Data availability* describing the accession number under which the data have been archived. This is typical for any genome paper. The genome assembly is deposited under the accession ID `QUAD00000000`, with the particular *version* of the genome assembled in this paper given the ID QUAD01000000. Future assemblies of the same data, perhaps using better software, could be uploaded and associated with this project, but given new accession IDs. This accession identifier can be used to query the assembly and retrieve the contigs which are labeled QUAD01000001-QUAD01000063. Follow the link in the paper to the genome project page [https://www.ncbi.nlm.nih.gov/nuccore/QUAD00000000](https://www.ncbi.nlm.nih.gov/nuccore/QUAD00000000). There you can find additional links to the Project accession, or the BioSample accession, all of which can tell you more about the project and how to access the data. As described in the Data Availability section of the paper, there were 10 other strains of Bd also sequenced in this study (using Illumina short read data) which are also linked under the same Project ID (PRJNA483086), but are not linked under the same Genome Assembly (QUAD00000000). By following links on the project page we can find pages for the sequenced individuals (SRXs) and sequencing runs (SRRs). I know, the accession IDs are hard to keep straight. These last IDs link to the actual sequence data files. For example, this page has details about the PacBio data we will be assembling: [https://www.ncbi.nlm.nih.gov/sra/SRX4676189[accn]](https://www.ncbi.nlm.nih.gov/sra/SRX4676189[accn]). 

Large fastq sequence data files are stored online in a weird format called `.sra` which includes a lot of connected metadata. This can be a pain when you just want to access the fastq files. We have installed a suite of command-line tools called the `sra-tools`, that allow you to query these databases using accession IDs, download the sequence data, and convert it into the proper format all in one command. From these tools the only one we really need is called  `fastq-dump`. The commands below will download the PacBio data for Bd, and Illumina data for one of the other strains included in the study which was shown to be identical to the one sequenced with PacBio. The PacBio data is all single reads, whereas the Illumina data is paired-end reads (separate R1 and R2 files).

*This download would take about 20 minutes*, so instead we downloaded the data files ahead of time for you. Therefore the code is commented out. 

In [4]:
%%bash

## Download Bd RTP6 PacBio reads
#fastq-dump SRR7825134

## Download Bd RTP5 Illumina HiSeq 2000 paired-end reads
#fastq-dump SRR7825135 --split-files


### The fastq data files
You have seen fastq data files before, and the PacBio data is no different from Illumina fastq data files, except the reads are just longer. We'll return to the PacBio reads in the next notebook, but for now we will focus on the Illumina short read data. How good of an assembly of the *Bd* genome can we get from short read data alone? 

<div class="alert alert-success">
    <b>[2] Question:</b> 
    How many sequenced reads are in each of the raw fastq data files (SRR7825134.fasta, SRR7825135_1.fasta and SRR7825135_2.fasta)? Use any method (bash or Python) to calculate and print the answer in the cell below. 
</div>

In [9]:
# your answer here


### Trim reads for quality and adapters

Filtering and trimming is an important step for any genome assembly. The cleaner your reads are the better and faster the genome will assemble. The command below uses the program `cutadapt` to trim Illumina adapters from the reads and trim from the 3' end when quality falls below a set threshold based on the quality scores in the fastq data files. To keep our notebook easy to read we will suppress the outputs printed to screen by `cutadapt`, which includes stats about how many reads were trimmed or removed, and write those stats to a report file instead. You can tell from the size difference between the trimmed and untrimmed files though that it a fair amount of the data was cleaned up. The trimming step here takes only a few minutes, but again, we've commented it out for effiency. 

We will not clean up the PacBio data files for now. These are expected to have lower quality scores, and so instead of trimming them to throw away the bad parts it is more common to try to correct the errors either as a first step before assembly, or during assembly. This usually involves mapping short-read data to the long reads. An example program for this is the tool `proovread`, but there are several alternatives. Again, we're going to skip this step here. 


In [14]:
%%bash

# run cutadapt trimming
# -q: trim 3' when quality < 10
# -a: trim adapters from R1
# -A: trim adapters from R2
# -o: output R1 filename
# -p: output R2 filename
cutadapt \
    -q 10 \
    -a AGATCGGAAGAGC -A AGATCGGAAGAGC \
    -o SRR7825135_1.trim.fastq \
    -p SRR7825135_2.trim.fastq \
    SRR7825135_1.fastq \
    SRR7825135_2.fastq > trim-report.txt

The report produced by `cutadapt` includes statistics about how many reads were trimmed based on low quality or adapter contamination. Open the report at [this link](...) to see the results. In the command above we named the new trimmed read files `SRR7825135_1.trim.fastq` and `SRR7825135_2.trim.fastq`. These are the files we will use going forward. 

<div class="alert alert-success">
    <b>[3] Question:</b> 
    From the statistics in the cutadapt report (see link above), how many bp were trimmed from reads in this dataset, and how many reads were excluded completely? Answer in markdown below. 
</div>

In [15]:
# your answer here


### Genome size and complexity
Your assigned reading by Li and Harkess titled "A guide to sequence your favorite plant genomes" describes some particular difficulties of assembling plant genomes, but most of these can be generalized to other organisms with difficult to assemble genomes as well. They make the important point that before starting any genome sequencing project it is important to learn as much as you can about your organism, as this should inform the type of approach that you take, and your expectations about the result you will get. Similarly, one of the first things you can do when you get some initial shotgun sequenced Illumina data is to investigate some of these same questions: How big is the genome, how variable, and what ploidy (has it experienced genome duplications)? It turns out we can learn answers to these questions by examining the distribution of kmer counts in the raw read data itself, without even having to map, align, or assemble it. 

<div class="alert alert-success">
    <b>[3] Question:</b> 
    In Figure 2 in Li and Harkess what characteristics of the kmer distribution plots are informative about heterozygosity and repetitiveness? Why does one distribution have one hump while the other has two? Answer in Markdown below.     
</div>

In [85]:
# answer here:

### Jellyfish: kmer counting tool
The [jellyfish](https://github.com/gmarcais/Jellyfish/tree/master/doc) program is a tool used to efficiently count kmers from a genome fasta file or fastq sequenced read files. It works much faster than the Python code we wrote last week to find all kmers in a sequence and is quite easy to use. It turns out we can learn a lot from examining the distribution of kmer counts. The quote below is from the Jellyfish paper by the authors, Marçais and Kingford (2011): 
    
<blockquote>

Given a string S, we are often interested in counting the number of occurrences in S of every substring of length k. These length-k substrings are called k-mers and the problem of determining the number of their occurrences is called k-mer counting.

Counting the k-mers in a DNA sequence is an important step in many applications. For example, genome assemblers using the overlap-layout-consensus paradigm, such as the Celera (Miller et al., 2008; Myers et al., 2000) and Arachne (Jaffe et al., 2003) assemblers, use k-mers shared by reads as seeds to find overlaps. Statistics on the number of occurrences of each k-mer are first computed and used to filter out which k-mers are used as seeds. Such k-mer count statistics are also used to estimate the genome size: if a large fraction of k-mers occur c times, we can estimate the sequencing coverage to be approximately c and derive an estimate of the genome size from c and the total length of the reads. In addition, in most short-read assembly projects, errors are corrected in the sequencing reads to improve the quality of the final assembly. For example, Kelley et al. (2010) use k-mer frequencies to assess the likelihood that a misalignment between reads is a sequencing error or a genuine difference in sequence. A third application is the detection of repeated sequences, such as transposons, which play an important biological role. De novo repeat annotation techniques find candidate regions based on k-mer frequencies (Campagna et al., 2005; Healy et al., 2003; Kurtz et al., 2008; Lefebvre et al., 2003). The counts of k-mers are also used to seed fast multiple sequence alignment (Edgar, 2004). Finally, k-mer distributions can produce new biological insights directly. Sindi et al. (2008) used k-mers frequencies with large k (20 ≤ k ≤ 100) to study the mechanisms of sequence duplication in genomes.
</blockquote>

Running jellyfish on the raw data files produces a `.jf` formatted file with information about the kmers in the data. Here we run jellyfish on the full size data files because kmer counting requires ~30X or more to show good peaks. This will take about 5 minutes to run. We can then extract a histogram from this file using the `histo` function from jellyfish, which  returns the histogram as a text table showing the number of kmers that occurred N times. You can see many occurred very few times or only once, then it decreases before beginning to increase again. 

In [ ]:
%%bash

# call jellyfish on short-read fastq files
# -m kmer size
# -s genome size estimate
# -t number of threads to use
# -C switch to count 'canonical kmers' (reverse complement)
# -o output location
# <(...) this decompresses the file as it is passed to jellyfish

# run jellyfish counting for k=21mer
jellyfish count \
    -m 21 -s 40M -t 2 -C \
    -o 21mer.jf \
    SRR7825135_1.fastq \
    SRR7825135_2.fastq
    
# write histogram of kmer counts to a file 
jellyfish histo 21mer.jf > 21mer.hist.csv

# print first 20 lines from histogram file to the screen
head -n 20 21mer.hist.csv


Let's now use Python to load the kmer count data with the Pandas library and plot it with the Toyplot library. We can see clearly that there is not best peak around at around 55X coverage or so. We will assume that this peak represents a single copy of the genome, and that any other peaks represent heterozygosity, errors, repetitions, and other such noise. So the density of our single-copy peak stretches from about 30 to 80. 

In [ ]:
# load the histogram data as a table
df = pd.read_csv("21mer.hist.csv", sep=" ", names=["bins", "counts"])
toyplot.fill(
    df.counts[2:200], 
    width=500, 
    height=300,
    ylabel="number of unique kmers", 
    xlabel="kmer coverage depth",
);

### Genome size estimation
We can estimate the genome size by taking the total number of kmers (multiple the coverage times the number of unique kmers in each bin) and dividing this by the mean coverage (peak position). We get an estimate of ~30Mb, which is close to the current estimated value of the *Bd* genome of 24Mb.  

In [ ]:
# calculate genome size estimate
np.sum(df.counts[2:] * df.bins[2:]) / 55

### *de novo* assembly methods

There are many programs available today for genome assembly. They vary in terms of requirements of the input data, their speed, RAM requirements, and the resulting accuracy of the assemblies. For some types of data, or sizes of genomes some tools will work better than others. Because *de novo* assembly is such a time consuming process it is difficult to compare many tools and so people often just use a program that others suggest. It's worth reading a few bioinformatics articles that compare assembly tools to learn more about which one might be best for your project. You optional assigned reading by Giordano et al. is one such paper. It compares several assembly tools on a variety of data sets. The tool we will use below, [spades](http://cab.spbu.ru/software/spades/), has been popular for years, and continually updated to keep pace with advancing sequencing technologies. It is a *de Bruijn* graph based assembly method that is best for short-read assembly, but which recently added support for *hybrid* assemblies that combine short-read contig assembly with scaffolding by using long-reads. We will try these approaches here. 

Because this genome and dataset are relatively small the spades assembly should complete in about 20 minutes or less. Note, we actually have *very* little resources available to us on codio, and this assembly could run much faster if it had more threads to compute on and more RAM available. 

### *de novo* assembly with spades: Illumina only

In [ ]:
%%bash

spades.py \
    --only-assembler \
    -o assembly_spades/ \
    -1 SRR7825135_1.trim.1M.fastq \
    -2 SRR7825135_2.trim.1M.fastq \
    -m 2 \
    -k 21


### *de novo* HYBRID assembly with spades: Illumina & PacBio

In [ ]:
%%bash

spades.py \
    --only-assembler \
    -o assembly_spades/ \
    -1 SRR7825135_1.trim.1M.fastq \
    -2 SRR7825135_2.trim.1M.fastq \
    --pacbio SRR7825134.100K.fastq \
    -m 2 \
    -k 21


### Success, you just assembled a genome, twice!


<div class="alert alert-success">
    <b>[4] Question:</b> 
    Look at the output written to screen in each of the spades assemblies. 
</div>

### Examine assembly quality using QUAST
The [quast] program is used to calculate genome quality statistics. It calculates N50 contig and scaffold sizes (something we've done before), and it reports BUSCO scores (number of single-copy genes present) which we've read about before -- or at least it would if we had annotated our genome and entered a GFF file. We'll put that off for another time for now.

In [ ]:
%%bash

# download the busco gene set to this computer
#quast-download-busco

# run quast on the Illumina assembled genome
quast.py -o results_assembly_1 assembly_spades_1/scaffolds.fasta 

# run quast on the Hybrid assembled genome
quast.py -o results_assembly_2 assembly_spades_2/scaffolds.fasta 


### After running quast
After running quast you can open a summary of the results in HTML format from the output directory. I've provided a link to it for you that should open in a new browser tab. 

+ [de novo Illumina assembly statistics](./results_assembly_1/latest/report.html)
+ [de novo hybrid Illumina/PacBio assembly statistics](./results_assembly_2/latest/report.html)

<div class="alert alert-success">
    <b>[5] Question:</b> 
    Which assembly do you think is better? Which statistics do you base this on?
</div>

<div class="alert alert-success">
    <b>[6] Question:</b> 
    Name three possible next steps you could take to improve the genome assembly?
</div>

### Question from last time: What are PacBio 'subreads'

Found answer here: http://seqanswers.com/forums/showthread.php?t=34790


Subreads are the individual sequence reads that are determined in real time from a template on the sequencer. These would correspond to the stretch of DNA between the adapter sequence.

CCS reads are the result of doing a consensus base calling from subreads that are all from the same template. If the template was short enough, then the polymerase will loop entirely from the beginning back to the beginning, sequencing the adapter before it starts on the template again. In this case, the same template piece of DNA is sequenced more than once, which means that the sequence data generated from each pass can be used to determine one single consensus sequence with higher base quality than the raw subreads. Thus, the quality distribution that you see is correct between the subreads and the CCS reads

